In [14]:
import findspark
import os
spark_location='spark-2.4.4-bin-hadoop2.7'
java8_location= '/usr/lib/jvm/java-8-openjdk-amd64' 
os.environ['JAVA_HOME'] = java8_location
findspark.init('/spark-2.4.4-bin-hadoop2.7/./bin/spark-submit') 

IndexError: list index out of range

In [12]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType, DoubleType
from pyspark.sql.functions import col, udf
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

kindle_store = spark.read.json("kindle_store.json")

PermissionError: [Errno 13] Permission denied: 'spark-2.4.4-bin-hadoop2.7/./bin/spark-submit'

In [17]:
kindle_store.write.parquet("kindle_store.parquet")

NameError: name 'kindle_store' is not defined

In [18]:
kindle_store = spark.read.parquet("kindle_store.parquet")

NameError: name 'spark' is not defined

In [19]:
reviews = kindle_store[["reviewText","overall"]]

NameError: name 'kindle_store' is not defined

In [20]:
reviews.show(20)


NameError: name 'reviews' is not defined

In [21]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

NameError: name 'sc' is not defined

In [ ]:
sqlContext.registerDataFrameAsTable(reviews, "table2")
reviews1 = sqlContext.sql("SELECT reviewText, overall from table2 LIMIT 50000")

In [ ]:
reviews1.show(20)


In [ ]:
#positive->1
#negative->0
def transform(star):
    if star >=3.0:
        return 1.0
    else:
        return 0.0
transformer = udf(transform)

In [ ]:
df = reviews1.withColumn("label", transformer(reviews['overall']))

In [ ]:
sqlContext.registerDataFrameAsTable(df, "table1")
df2 = sqlContext.sql("SELECT reviewText, label from table1 WHERE reviewText != ''")

In [ ]:
(training, test) = df2.randomSplit([0.9, 0.1])

In [ ]:
training.show(12)

In [ ]:
training

In [ ]:
regexTokenizer = RegexTokenizer(inputCol="reviewText", outputCol="words", pattern="\\W")
# alternatively, pattern="\\w+", gaps(False)
countTokens = udf(lambda words: len(words), IntegerType())

In [ ]:
regexTokenized = regexTokenizer.transform(training)
regexTokenized.select("reviewText", "words") \
    .withColumn("tokens", countTokens(col("words"))).show(truncate=False)

In [ ]:
regexTokenized.head()

In [ ]:
htf = HashingTF(inputCol="words", outputCol="features")
tf = htf.transform(regexTokenized)
train = tf[["label","features"]]

In [ ]:
types = [f.dataType for f in train.schema.fields]
types

In [ ]:
train2 = train.withColumn("label",train["label"].cast(DoubleType()))
types = [f.dataType for f in train2.schema.fields]
types

In [ ]:
train2

In [ ]:
lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)
pipeline = Pipeline(stages=[lr])


In [ ]:
model = pipeline.fit(train2)

In [22]:
test.show()

NameError: name 'test' is not defined

In [ ]:
regexTokenized_test = regexTokenizer.transform(test)
regexTokenized.select("reviewText", "words") \
    .withColumn("tokens", countTokens(col("words"))).show(truncate=False)

In [23]:
tf_test = htf.transform(regexTokenized_test)
testSet = tf_test[["label","features"]]
testSet = testSet.withColumn("label",testSet["label"].cast(DoubleType()))

NameError: name 'htf' is not defined

In [ ]:
prediction = model.transform(testSet)

In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(prediction)
print("Test Error = %g " % (1.0 - accuracy))

In [24]:
sc.stop()

NameError: name 'sc' is not defined